In [0]:
# lesson 32 Tain-Val-Test 划分

In [0]:
 import torch 
 import torch.nn as nn 
 import torch.nn.functional as F
 import torch.optim as optim 
 from torchvision import datasets, transforms 

In [0]:
batch_size= 100 
learning_rate = 0.001 
epochs = 500 

In [0]:
train_db = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)

test_db = datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
test_loader = torch.utils.data.DataLoader(test_db,
    batch_size=batch_size, shuffle=True)

In [0]:
print('train: ', len(train_db))
print('test: ', len(test_db))

In [0]:
train_db, val_db = torch.utils.data.random_split(train_db,[50000,10000])

print('train: ', len(train_db))
print('validation: ', len(val_db))

In [0]:
train_loader = torch.utils.data.DataLoader(
    train_db, batch_size = batch_size, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    val_db, batch_size= batch_size, shuffle = True
)

print('train: ', len(train_db))
print('validation: ', len(val_db))

In [0]:
# module 

class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200,200),
            nn.LeakyReLU(inplace =True),
            nn.Linear(200,10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self,x):
        x = self.model(x)
        return x 

In [0]:
torch.cuda.is_available()

In [0]:
# use GPU 

device = torch.device('cuda:0')
net = MLP().to(device)

optimizer = optim.SGD(net.parameters(), lr= learning_rate)
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
for epoch in range(epochs):
    for index, (data, target) in enumerate(train_loader):
        data = data.view(-1,28*28)

        data, target = data.to(device), target.to(device)

        logits = net(data)

        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if index %10 ==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, index * len(data), len(train_loader.dataset),
                       100. * index / len(train_loader), loss.item()))
            

    test_loss =0 
    correct =0 

    for data, target in val_loader:
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.to(device)

        logits = net(data)
        test_loss += criterion(logits, target).item()

        pred = logits.max(dim=1)[1]

        correct += pred.eq(target.data).sum()

    test_loss /= len(val_loader.dataset)

    print('\nVAL set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))